In [1]:
param_x_api_key = '2BeBXg69osbuG45Rjtv1wLEnHL3nuT3WPQQK1EaYHT4PutPkDrXB'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                data = '''{"getArticles":
                {"articleCountry":"DE",
                 "provider":40000,
                 "lang":"en",
                 "searchQuery":"''' + input_.loc[a, 'Row_Id'].split(']-[')[1].replace(']', '') + '''",
                 "searchMatchType":"exact",
                 "searchType":0,
                 "page":1,
                 "perPage":10,
                 "filterQueries":["(dataSupplierId NOT IN (4978,4982))"],
                 "dataSupplierIds":[''' + input_.loc[a, 'Row_Id'].split(']-[')[0].replace('[', '') + '''],
                 "genericArticleIds":[''' + input_.loc[a, 'Groups'] + '''],
                 "includeAll":false,
                 "includeLinkages":true,
                 "linkagesPerPage":100,
                 "includeGenericArticles":true,
                 "includeArticleCriteria":true,
                 "includeMisc":true,
                 "includeImages":true,
                 "includePDFs":true,
                 "includeLinks":true,
                 "includeArticleText":true,
                 "includeOEMNumbers":true,
                 "includeReplacedByArticles":true,
                 "includeReplacesArticles":true,
                 "includeComparableNumbers":true,
                 "includeGTINs":true,
                 "includeTradeNumbers":true,
                 "includePrices":true,
                 "includePartsListArticles":false,
                 "includeAccessoryArticles":false,
                 "includeArticleLogisticsCriteria":true,
                 "includeDataSupplierFacets":false,
                 "includeGenericArticleFacets":false,
                 "includeCriteriaFacets":false}}'''

                headers = UA.get_User_Agent_Requests()
                headers['X-Api-Key'] = param_x_api_key

                resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                     data=data,
                                     headers=headers,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                part_number = json_data['articles'][0]['articleNumber'].strip()
                
                # = = = = = = = = = = = = = = =
                
                manufactuer = json_data['articles'][0]['mfrName'].strip()
                
                # = = = = = = = = = = = = = = =
                
                type_ = json_data['articles'][0]['genericArticles'][0]['genericArticleDescription'].strip()

                # = = = = = = = = = = = = = = =
                
                article_id = str(json_data['articles'][0]['genericArticles'][0]['legacyArticleId'])

                # = = = = = = = = = = = = = = =
                
                information = ' '.join(json_data['articles'][0]['articleText'][0]['text'])

                # = = = = = = = = = = = = = = =
                
                gtin = '' if len(json_data['articles'][0]['gtins']) == 0 else json_data['articles'][0]['gtins'][0].strip()

                # = = = = = = = = = = = = = = =
                
                oe = ';'.join(sorted(list(set(oe['articleNumber'].strip().replace(' ', '') for oe in json_data['articles'][0]['oemNumbers']))))

                # = = = = = = = = = = = = = = =
                
                list_manufactuer_id = [''] if len(sorted(list(set([str(oe['mfrId']) for oe in json_data['articles'][0]['oemNumbers']])))) == 0 else sorted(list(set([str(oe['mfrId']) for oe in json_data['articles'][0]['oemNumbers']])))

                # = = = = = = = = = = = = = = =
                
                list_criteria = json_data['articles'][0]['articleCriteria']
                
                # = = = = = = = = = = = = = = =
                
                src = '' if len(json_data['articles'][0]['images']) == 0 else json_data['articles'][0]['images'][0]['imageURL3200'].strip()

                # = = = = = = = = = = = = = = =
                
                price_type = '' if len(json_data['articles'][0]['prices']) == 0 else json_data['articles'][0]['prices'][0]['kindOfPriceDescription'].strip()

                # = = = = = = = = = = = = = = =
                
                price_unit = '' if len(json_data['articles'][0]['prices']) == 0 else json_data['articles'][0]['prices'][0]['priceUnitDescription'].strip()

                # = = = = = = = = = = = = = = =
                
                quantity_unit = '' if len(json_data['articles'][0]['prices']) == 0 else json_data['articles'][0]['prices'][0]['quantityUnitDescription'].strip()

                # = = = = = = = = = = = = = = =
                
                price = '' if len(json_data['articles'][0]['prices']) == 0 else json_data['articles'][0]['prices'][0]['currencyCode'].strip() + ' ' + str(json_data['articles'][0]['prices'][0]['price'])

                # = = = = = = = = = = = = = = =
                
                discount_group = '' if len(json_data['articles'][0]['prices']) == 0 else json_data['articles'][0]['prices'][0]['isDiscount']

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'序号': int(input_.loc[a, 'No.']),
                                        'Manufactuer': manufactuer,
                                        'Part_Number': part_number,
                                        'Type': type_,
                                        'GTIN': gtin,
                                        'Price_Type': price_type,
                                        'Price_Unit': price_unit,
                                        'Quantity_Unit': quantity_unit,
                                        'Price': price,
                                        'Discount_Group': discount_group,
                                        'Vehicle_1': '',
                                        'Vehicle_2': '',
                                        'OE': oe,
                                        'Pic': '',
                                        'Url': 'https://web.tecalliance.net/tecdocsw/en/parts/' + input_.loc[a, 'Row_Id'].split(']-[')[0].replace('[', '') + '/' + input_.loc[a, 'Row_Id'].split(']-[')[1].replace(']', '') + '/detail',
                                        'Src': src,
                                        'Information': information,
                                        'Article_Id': article_id,
                                        'Manufactuer_Id': list_manufactuer_id})

                for criteria in list_criteria:
                    df_temp['Criteria_' + criteria['criteriaAbbrDescription'] if 'criteriaAbbrDescription' in criteria else 'Criteria_' + criteria['criteriaDescription'].strip()] = criteria['formattedValue'].strip() + criteria['criteriaUnitDescription'].strip() if 'criteriaUnitDescription' in criteria else criteria['formattedValue'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Row_Id': input_.loc[a, 'Row_Id'],
                                     'Groups': input_.loc[a, 'Groups']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Row_Id'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
list_column = output_correct.columns.tolist()
list_column.remove('Manufactuer_Id')
output_correct_1 = output_correct[list_column].drop_duplicates(ignore_index=True)
output_correct_1 = output_correct_1.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct_1.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
output_correct_2 = output_correct[['Article_Id', 'Manufactuer_Id']].drop_duplicates(ignore_index=True)
output_correct_2 = output_correct_2[output_correct_2['Manufactuer_Id'] != '']
output_correct_2 = output_correct_2.sort_values(by=['Article_Id', 'Manufactuer_Id'], ascending=[True, True]).reset_index(drop=True)
output_correct_2.to_excel('./part_2-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：413

[ok] - [101]-[01513]
[尝试次数：1] - [剩余数量：393] - [当前时间：14:18:03]

[ok] - [101]-[02024]
[尝试次数：1] - [剩余数量：392] - [当前时间：14:18:03]

[ok] - [101]-[01628]
[尝试次数：1] - [剩余数量：391] - [当前时间：14:18:04]

[ok] - [101]-[02222]
[尝试次数：1] - [剩余数量：390] - [当前时间：14:18:04]

[ok] - [101]-[02034]
[尝试次数：1] - [剩余数量：389] - [当前时间：14:18:04]

[ok] - [101]-[02032]
[尝试次数：1] - [剩余数量：388] - [当前时间：14:18:04]

[ok] - [101]-[01514]
[尝试次数：1] - [剩余数量：387] - [当前时间：14:18:04]

[ok] - [101]-[01520]
[尝试次数：1] - [剩余数量：386] - [当前时间：14:18:04]

[ok] - [101]-[01626]
[尝试次数：1] - [剩余数量：385] - [当前时间：14:18:04]

[ok] - [101]-[02023]
[尝试次数：1] - [剩余数量：384] - [当前时间：14:18:04]

[ok] - [101]-[02031]
[尝试次数：1] - [剩余数量：383] - [当前时间：14:18:04]

[ok] - [101]-[02388]
[尝试次数：1] - [剩余数量：382] - [当前时间：14:18:04]

[ok] - [101]-[03048]
[尝试次数：1] - [剩余数量：381] - [当前时间：14:18:04]

[ok] - [101]-[01110]
[尝试次数：1] - [剩余数量：380] - [当前时间：14:18:04]

[ok] - [101]-[01750]
[尝试次数：1] - [剩余数量：379] - [当前时间：14:18:04]

[ok] - [101]-[04125]
[尝试次数：1] - [剩余数量：378] - [当前时间：14:18:04]


[ok] - [101]-[172797]
[尝试次数：1] - [剩余数量：260] - [当前时间：14:18:21]

[ok] - [101]-[170838]
[尝试次数：1] - [剩余数量：259] - [当前时间：14:18:21]

[ok] - [101]-[172292]
[尝试次数：2] - [剩余数量：258] - [当前时间：14:18:22]

[ok] - [101]-[173218]
[尝试次数：1] - [剩余数量：257] - [当前时间：14:18:22]

[ok] - [101]-[15928]
[尝试次数：6] - [剩余数量：256] - [当前时间：14:18:22]

[ok] - [101]-[15911]
[尝试次数：9] - [剩余数量：255] - [当前时间：14:18:22]

[ok] - [101]-[173815]
[尝试次数：1] - [剩余数量：254] - [当前时间：14:18:22]

[ok] - [101]-[05651]
[尝试次数：1] - [剩余数量：253] - [当前时间：14:18:22]

[ok] - [101]-[15932]
[尝试次数：11] - [剩余数量：252] - [当前时间：14:18:24]

[ok] - [101]-[173814]
[尝试次数：4] - [剩余数量：251] - [当前时间：14:18:24]

[ok] - [101]-[12019]
[尝试次数：4] - [剩余数量：250] - [当前时间：14:18:24]

[ok] - [101]-[173216]
[尝试次数：6] - [剩余数量：249] - [当前时间：14:18:24]

[ok] - [101]-[175790]
[尝试次数：2] - [剩余数量：248] - [当前时间：14:18:25]

[ok] - [101]-[175984]
[尝试次数：1] - [剩余数量：247] - [当前时间：14:18:25]

[ok] - [101]-[173673]
[尝试次数：1] - [剩余数量：246] - [当前时间：14:18:26]

[ok] - [101]-[175344]
[尝试次数：1] - [剩余数量：245] - [当前时间：14:18:2

[ok] - [101]-[24585]
[尝试次数：1] - [剩余数量：127] - [当前时间：14:18:59]

[ok] - [101]-[26610]
[尝试次数：1] - [剩余数量：126] - [当前时间：14:18:59]

[ok] - [101]-[24583]
[尝试次数：1] - [剩余数量：125] - [当前时间：14:18:59]

[ok] - [101]-[26383]
[尝试次数：1] - [剩余数量：124] - [当前时间：14:18:59]

[ok] - [101]-[24191]
[尝试次数：1] - [剩余数量：123] - [当前时间：14:18:59]

[ok] - [101]-[23988]
[尝试次数：1] - [剩余数量：122] - [当前时间：14:19:00]

[ok] - [101]-[26776]
[尝试次数：1] - [剩余数量：121] - [当前时间：14:19:00]

[ok] - [101]-[26308]
[尝试次数：1] - [剩余数量：120] - [当前时间：14:19:00]

[ok] - [101]-[24029]
[尝试次数：1] - [剩余数量：119] - [当前时间：14:19:01]

[ok] - [101]-[24009]
[尝试次数：1] - [剩余数量：118] - [当前时间：14:19:01]

[ok] - [101]-[27417]
[尝试次数：1] - [剩余数量：117] - [当前时间：14:19:02]

[ok] - [101]-[26666]
[尝试次数：1] - [剩余数量：116] - [当前时间：14:19:02]

[ok] - [101]-[28198]
[尝试次数：1] - [剩余数量：115] - [当前时间：14:19:02]

[ok] - [101]-[28202]
[尝试次数：1] - [剩余数量：114] - [当前时间：14:19:02]

[ok] - [101]-[26478]
[尝试次数：1] - [剩余数量：113] - [当前时间：14:19:02]

[ok] - [101]-[27420]
[尝试次数：1] - [剩余数量：112] - [当前时间：14:19:03]

[ok] - [

[ok] - [101]-[49215]
[尝试次数：5] - [剩余数量：0] - [当前时间：14:19:30]

[ok] - [101]-[44496]
[尝试次数：3] - [剩余数量：0] - [当前时间：14:19:32]

[ok] - [101]-[15527]
[尝试次数：3] - [剩余数量：0] - [当前时间：14:19:34]

[ok] - [101]-[44423]
[尝试次数：5] - [剩余数量：0] - [当前时间：14:19:36]

[ok] - [101]-[173635]
[尝试次数：3] - [剩余数量：0] - [当前时间：14:19:39]

[ok] - [101]-[174193]
[尝试次数：3] - [剩余数量：0] - [当前时间：14:19:39]

[ok] - [101]-[171789]
[尝试次数：2] - [剩余数量：0] - [当前时间：14:19:41]

[ok] - [101]-[43721]
[尝试次数：1] - [剩余数量：0] - [当前时间：14:19:53]

[ok] - [101]-[07654]
[尝试次数：3] - [剩余数量：0] - [当前时间：14:19:55]

[ok] - [101]-[32281]
[尝试次数：2] - [剩余数量：0] - [当前时间：14:20:31]

[ok] - [101]-[45608]
[尝试次数：2] - [剩余数量：0] - [当前时间：14:20:42]

[ok] - [101]-[44371]
[尝试次数：5] - [剩余数量：0] - [当前时间：14:21:55]

输出ing...

Done ~
